<h1>Part 1 </h1>
<h2> Web scraping</h2>

In [7]:
  !pip install watson-developer-cloud==1.5

     |████████████████████████████████| 225kB 24.5MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 25.5MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 36.2MB/s eta 0:00:01
     |████████████████████████████████| 2.7MB 33.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 26.2MB/s eta 0:00:01
     |████████████████████████████████| 235kB 34.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 27.8MB/s eta 0:00:01
     |████████████████████████████████| 163kB 40.9MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/81/cd/de/4e0916f623c2d125502e493394fd333ed693960264d4b7e524
Successfully built watson-developer-cloud
  Found existing installation: cryptography 2.5
    Uninstalling cryptography-2.5:
      Successfully uninstalled cryptography-2.5
  Found existing installation: attrs 18.2.0
    Uninstalling attrs-18.2.0:
      Successfully uninstalled attrs-18.2.0


In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml.html as lh

In [9]:
from urllib.request import urlopen

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [11]:
try:
   page = urlopen(url)
except:
   print("Error opening the URL")

In [12]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

In [13]:
#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)


In [14]:

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:

canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
canada_df["Neighborhood"] = canada_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

In [17]:
if(canada_df.index.name != 'Postal Code'):
    canada_df = canada_df.set_index('Postal Code')
    
canada_df.head()


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:

canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
canada_df.shape

(103, 2)

In [23]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 931kB 10.9MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 33.1MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 35.2MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 31.1MB/s eta 0:00:01


In [26]:
locator = Nominatim(user_agent="myGeocoder")

In [32]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
canada_df['location'] = canada_df['Borough'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
canada_df['point'] = canada_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
canada_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(canada_df['point'].tolist(), index=canada_df.index)


In [33]:
canada_df.head()

,Borough,Neighborhood,location,point,latitude,longitude,altitude
Postal Code,,,,,,,
M3A,North York,Parkwoods,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
M4A,North York,Victoria Village,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
M5A,Downtown Toronto,"Regent Park, Harbourfront","(CF Toronto Eaton Centre, Yonge Street, Downto...","(43.6541737, -79.38081164513409, 0.0)",43.654174,-79.380812,0.0
M6A,North York,"Lawrence Manor, Lawrence Heights","(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","(CF Toronto Eaton Centre, Yonge Street, Downto...","(43.6541737, -79.38081164513409, 0.0)",43.654174,-79.380812,0.0


In [35]:
canada_df.drop(columns =['location', 'point' , 'altitude']) 


,Borough,Neighborhood,latitude,longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.754326,-79.449117
M4A,North York,Victoria Village,43.754326,-79.449117
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.654174,-79.380812
M9A,Etobicoke,Islington Avenue,43.671459,-79.552492
M1B,Scarborough,"Malvern, Rouge",54.282001,-0.401187
M3B,North York,Don Mills,43.754326,-79.449117
M4B,East York,"Parkview Hill, Woodbine Gardens",39.973709,-76.686355


In [41]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [43]:
import folium

In [53]:
address = 'Toronto'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [59]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng , label in zip(canada_df['latitude'], canada_df['longitude'] , canada_df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [60]:
CLIENT_ID = 'WZ2WEHZLUNUUIUK25ZGO1VPPAWNFETZ4AVZNVWSEXK4O4ULX' 
CLIENT_SECRET = 'CUMEVMLZEOER5YHYDPKZDMAEAE1DSUFI3HJ5KIMZLIXGP11S' 
VERSION = '20190505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZ2WEHZLUNUUIUK25ZGO1VPPAWNFETZ4AVZNVWSEXK4O4ULX
CLIENT_SECRET:CUMEVMLZEOER5YHYDPKZDMAEAE1DSUFI3HJ5KIMZLIXGP11S


In [61]:
canada_df.loc[]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [0] of <class 'int'>